# 2 Delayed Markov Decision Processes

## 2.1 Naive markovian policies

To attempt writing fixed-point equations for $V^\pi(s)$ under the given delayed MDP with naive Markovian policies, we first consider the standard Bellman equation in a typical MDP without delays:


$$ V^\pi(s) = r(s, \pi(s)) + \lambda \sum_{s{\prime}} p(s{\prime} \mid s, \pi(s)) V^\pi(s{\prime})$$

In this equation, the value at state $s$ depends directly on the immediate reward received at $s$ and the expected future value $V^\pi(s{\prime})$ of the next state $s{\prime}$, given action $\pi(s)$.

In the delayed MDP scenario at hand, the agent’s action at time $t$ depends on the state from $d$ steps ago, $s_{t-d}$. Specifically, for $t \geq d+1$, the action is $a_t = \pi(s_{t-d})$. This leads to the following problems when trying to write a fixed-point equation for $V^\pi(s)$: 

1. The standard Bellman equation requires the future to be independent of the past given the present state and action, as defined in the lecture on the board or more formal on slide 11 from the first slide deck in the Markovian dynamics. In the given case the action at time $t$ depends on $s_{t-d}$, a past state, rather than the current state $s_t$, thus making the process non-Markovian with respect to $s_t$.

2. Due to the delayed dependence, the immediate reward $r(s_t, a_t)$ and the transition probabilities $p(s_{t+1} \mid s_t, a_t)$ depend on $s_{t-d}$ through $a_t$. This creates a situation where  V^\pi(s)  cannot be expressed solely in terms of  V^\pi(s{\prime})  for states  s{\prime}  reachable from  s  in one step. The recursive structure, as introduced on slide 24 part 1, required for solving a fixed-point equation is broken.






## 2.2 Equivalent augmented state MDP

Agumenting the states with $\bar{s}_t =(s_{t−d},a_{t−d},...,a_{t−1})$ reduces the problem to an MDP with no delay. By augmenting the states this way the MDP becomes Markovian with respect to the augmented state $\bar{s}_t$, thus solving the problems from 2.1. The additional states "absorb" the delays and makes it possible for the agent to operate in an MDP without delays, where standard reinforcement learning techniques can be applied.



At time t, the augmented state is:

$\bar{s}_t = (s_{t-d}, a_{t-d}, a_{t-d+1}, \ldots, a_{t-1})$

- $s_{t-d}$: The state observed d steps ago (the most recent state information available).
- $(a_{t-d}, \ldots, a_{t-1})$: The sequence of actions taken since observing $s_{t-d}$.

By including these elements, the augmented state  $\bar{s}_t$  contains all the necessary historical information to determine future transitions and rewards based solely on $\bar{s}_t$ and the current action $a_t$. 

When the agent takes action $a_t$ at time $t$, the next augmented state $\bar{s}_{t+1}$ becomes:

$\bar{s}_{t+1} = (s_{t+1 - d}, a_{t+1 - d}, \ldots, a_t)$


The transition probability $P(\bar{s}_{t+1} \mid \bar{s}_t, a_t)$ is determined by:
1. State component update: $s_{t-d} = s_{t+1 - d}$ with transition probability: 
$P(s_{t - d + 1} \mid s_{t - d}, a_{t - d})$
- This comes from the original MDP’s state transition probabilities and we assume that it is given 
2. Action Components Update: $(a_{t+1 - d}, \ldots, a_t) = (a_{t - d + 1}, \ldots, a_t)$
- Deterministic Update: The actions  $a_{t - d + 1}, \ldots, a_t$ are known from previous decisions.

### Constructing the Transition Probability:

The transition probability from  $\bar{s}_t$ to $\bar{s}_{t+1}$ given action $a_t$ is:


$$P(\bar{s}{t+1} \mid \bar{s}t, a_t) = P(s{t - d + 1} \mid s{t - d}, a_{t - d}) \times I\left(\text{Action components updated as expected} \right)$$

- $I(\cdot)$ is the indicator function, which equals 1 if the action components in $\bar{s}_{t+1}$ are updated exactly as per the deterministic rules, and 0 otherwise.

Since the action components always update deterministically (they are known and controlled by the agent), the indicator function $I(\cdot)$ is always 1 when considering valid transitions. Therefore, the transition probability simplifies to:

$$P(\bar{s}_{t+1} \mid \bar{s}_t, a_t) = P(s_{t - d + 1} \mid s_{t - d}, a_{t - d}) \times 1$$

This expression holds only for the valid transitions where action components update as expected; otherwise, the transition probability is zero.


### Reward Function 
The reward function in the augmented MDP is:

$r(\bar{s}_t, a_t) = r(s_{t-d}, a_{t-d})$

The immediate reward at time t depends on the delayed state $s_{t-d}$ and delayed action $a_{t-d}$, both components of $\bar{s}_t$.
The reward does not depend on the current action $a_t$ because the environment provides feedback based on actions taken d steps ago.
Therefore the reward at time $t$ corresponds to the action taken at time $t-d$.




Q8: In the equivalent MDP, consider a deterministic policy $\pi$ defined by $\pi(\bar{s})$ for all augmented state $\bar{s}$. Write the fixed point equations satisfied by $V^\pi$. Propose a RL algorithm to solve them.

Given the deterministic policy $\pi$ the value function $V^\pi(\bar{s})$ satisfies the following fixed-point equation (the Bellman equation for policy evaluation):
$$
V^\pi(\bar{s}) = r(\bar{s}, \pi(\bar{s})) + \gamma \sum_{\bar{s}{\prime}} P(\bar{s}{\prime} \mid \bar{s}, \pi(\bar{s})) V^\pi(\bar{s}{\prime})
$$

where:
- $\bar{s}$ is the current augmented state.
- $\pi(\bar{s})$ is the action taken in state  \bar{s}  according to the policy.
- $r(\bar{s}, \pi(\bar{s}))$ is the immediate reward received when taking action $\pi(\bar{s})$ in state $\bar{s}$.
- $\gamma$ is the discount factor (with $0 \leq \gamma < 1$).
- $P(\bar{s}{\prime} \mid \bar{s}, \pi(\bar{s}))$ is the transition probability from $\bar{s}$ to $\bar{s}{\prime}$ when action $\pi(\bar{s})$ is taken.


We can solve this fixed point equation by using a value iteration algorithm. This is possible because we have a complete model of the environment, as we know the stationary transition probabilities $p(\cdot | s, a)$ and the reward function according to the exercise.

The outline of the algorithm is as followed: 

- Start by initializing $V(\bar{s})$ for all states to some arbitrary values (e.g., zeros).
- Update $V(\bar{s})$ for each state based on the Bellman update equation:

$V(\bar{s}) = \max_a \left[ r(\bar{s}, a) + \gamma \sum_{\bar{s}{\prime}} P(\bar{s}{\prime} \mid \bar{s}, a) V(\bar{s}{\prime}) \right]$

- The loop continues until the maximum change in $V(\bar{s})$ between iterations, $\Delta$, is less than a small threshold $\theta$. 
- After the value function converges, we can extract an optimal policy $\pi^*$ by selecting actions that maximize the expected value at each state.

In the below pseudo code the following variables are used: 
- $\gamma$ is the discount factor, which determines how future rewards are valued relative to immediate rewards.
- $P(\bar{s}{\prime} \mid \bar{s}, a)$ represents the transition probability from state $\bar{s}$ to $\bar{s}{\prime}$ given action $a$.
- $r(\bar{s}, a)$ is the immediate reward received after taking action $a$ in state $\bar{s}$.


```
// Initialize V(𝑠̅) arbitrarily for all augmented states 𝑠̅
V = {}
for 𝑠̅ in all_possible_augmented_states:
    V[𝑠̅] = 0

// Set a threshold for convergence
θ = 0.0001  // A small positive number

// Value Iteration Loop:
while True:
    Δ = 0  // Initialize the maximum change in V to zero
    for 𝑠̅ in all_possible_augmented_states:
        v = V[𝑠̅]  // Store the current value of the state

        // Calculate the new value using the Bellman update equation
        V[𝑠̅] = max_a [r(𝑠̅, a) + γ * sum_{𝑠̅'} P(𝑠̅' | 𝑠̅, a) * V[𝑠̅']]

        // Update the maximum change in V
        Δ = max(Δ, abs(v - V[𝑠̅]))

    // Check for convergence
    if Δ < θ:
        break  // Exit the loop when the change is below the threshold

// Policy Extraction (optional, to derive an optimal policy π* from V)
π = {}
for 𝑠̅ in all_possible_augmented_states:
    // Select the action that maximizes the expected value
    π[𝑠̅] = argmax_a [r(𝑠̅, a) + γ * sum_{𝑠̅'} P(𝑠̅' | 𝑠̅, a) * V[𝑠̅']]
```

A Q-learning algorithm would also be an option to answer this question. However, it does not directly solve for the value function $V^\pi(\bar{s})$ as defined by the Bellman equation stated above. This is because Q-learning is an off-policy method that estimates the optimal action-value function $Q^*(\bar{s}, a)$. It updates the Q-values using the observed rewards and future estimated values but does not explicitly calculate $V^\pi(\bar{s})$. From the output of the Q algorithm we can infer the value function. We would use this method if the transition probabilities were unknown. 

```
// Initialize Q(𝑠̅, a) arbitrarily for all 𝑠̅ and a
Q = {} 
for s in all_possible_augmented_states:
    for a in all_possible_actions:
        Q[(s, a)] = 0 

// Episode Loop for e episodes:
For i in range(0, e):
    // Initialize the augmented state 𝑠̅ with dummy values
    𝑠̅ = (0, 0, ..., 0) // Initialize 𝑠̅ with dummy state and action history

    // Time Step Loop:
    While not end of episode:
        // 1. Action Selection:
        If random() < ε:
            aₜ = random_action()
        else:
            aₜ = argmax_a Q(𝑠̅, a)

        // 2. Environment Interaction:
        // Execute action aₜ
        r = r(sₜ₋d, aₜ₋d) // Observe reward based on delayed state and action
        s' = sample_next_state(sₜ₋d, aₜ₋d) // Observe next state based on transition probability

        // Update the augmented state:
        𝑠̅' = (s', aₜ₋(d-1), ..., aₜ₋₁, aₜ)

        // 3. Q-value Update:
        Q(𝑠̅, aₜ) = Q(𝑠̅, aₜ) + α * [r + λ * max_{a'} Q(𝑠̅', a') - Q(𝑠̅, aₜ)]

        // 4. State Transition:
        𝑠̅ = 𝑠̅'

    // Episode Termination:
    // End the episode after a fixed number of steps or when a terminal condition is met
// After running Q-learning, infer the optimal value function V*(𝑠̅)
For s in all_possible_augmented_states:
    V*(s) = max_a Q(s, a)
```